In [1]:
import numpy as np

from dataset_load import *

import pandas as pd

path = r'../../data/womens-ecommerce-clothing-reviews/Womens_Clothing_E-Commerce_Reviews.csv'

df = pd.read_csv(path)

X = list(df['Review Text'])
y = list(df['Rating'])
y_label = np.asarray(y)


y_label[y_label<3] = 0
y_label[y_label>3] = 1

neutral_indices = np.where(y_label==3)[0]
y_label = np.delete(y_label, neutral_indices)
X = np.delete(X, neutral_indices)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, ShuffleSplit

# split
X_train_split, X_test_split, y_train, y_test = train_test_split(X, y_label, test_size=0.33, random_state=42)

# preprocessing
X_train_corpus_update = update_corpus_contraction(X_train_split)
X_test_corpus_update = update_corpus_contraction(X_test_split)

(75, 2)
corpus update start
corpus update end

(75, 2)
corpus update start
corpus update end



In [2]:
####################################################################

# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

token = r"(?u)\b[\w\'/]+\b"
cv = CountVectorizer(min_df = 100, token_pattern=token, lowercase=True, binary=True)
X_train = cv.fit_transform(X_train_corpus_update)
X_test = cv.transform(X_test_corpus_update)

######################################################################

In [3]:
len(cv.get_feature_names())

720

### LR1

In [4]:
from sklearn.linear_model import LogisticRegression

lr1 = LogisticRegression(penalty='l1', random_state=42)
lr1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## LR2

In [5]:
word_list, connotation = load_unigrams('./ecom-unigrams.txt', X_train_corpus_update, y_train)

In [6]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_corpus_update, X_test_corpus_update, 
                                                          word_list, connotation)

In [7]:
htcv = CountVectorizer(min_df = 100, token_pattern=token, lowercase=True, binary=True, vocabulary=word_list)

In [8]:
ht_train = htcv.transform(X_train_corpus_update)
ht_test = htcv.transform(X_test_corpus_update)

In [9]:
y_train_agreement_pos = np.copy(y_train_agreement)
y_train_agreement_neg = np.copy(y_train_agreement)
y_test_agreement_pos = np.copy(y_test_agreement)
y_test_agreement_neg = np.copy(y_test_agreement)

y_train_agreement_pos[y_train_agreement_pos == -1] = 0
y_train_agreement_neg[y_train_agreement_neg == 1] = 0
y_test_agreement_pos[y_test_agreement_pos == -1] = 0
y_test_agreement_neg[y_test_agreement_neg == 1] = 0

#### From actual label

In [10]:
lr2_train = list()
for i,y in enumerate(y_train):
    if y == 1:
        lr2_train.append(y_train_agreement_pos[i] * y)
    else:
        lr2_train.append(y_train_agreement_neg[i] * y)
        
lr2_train = np.asarray(lr2_train)

In [11]:
lr2 = LogisticRegression(penalty='l1', random_state=42)
lr2.fit(lr2_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [39]:
# Test with predict {0,1}

lr1_predict = lr1.predict(X_test)

lr2_test_bin = list()
for i,y in enumerate(lr1_predict):
    if y==1:
        lr2_test_bin.append(y_test_agreement_pos[i] * y)
    else:
        lr2_test_bin.append(y_test_agreement_neg[i] * y)
    
lr2_test_bin = np.asarray(lr2_test_bin)

print(lr2.score(lr2_test_bin, y_test))

# Test with predict [0,1]

lr1_predict = lr1.predict_proba(X_test)[:,1]

lr2_test_proba = list()
for i,y in enumerate(lr1_predict):
    if y>=0.5:
        lr2_test_proba.append(y_test_agreement_pos[i] * y)
    else:
        lr2_test_proba.append(y_test_agreement_neg[i] * y)
    
lr2_test_proba = np.asarray(lr2_test_proba)

print(lr2.score(lr2_test_proba, y_test))
print(lr2.score(lr2_train,y_train))

0.8821108334558283
0.8821108334558283
0.8837966985230234


In [13]:
weight = lr2.coef_.flatten()
weight

array([2.07225276, 2.26577944, 2.17978121, 3.36649573, 3.63447905,
       5.39642824, 3.06199704, 3.67335775, 4.12306176, 2.01960772,
       2.17205167, 2.55062902, 4.96199048, 3.05388252, 5.87407455,
       4.16544223, 3.35259895, 2.16041677, 0.        , 0.        ,
       0.        , 0.        , 2.08388897, 1.50680413, 0.        ,
       2.67218354, 2.84222275, 2.67236185, 1.42215374, 0.67960568,
       1.39062592, 4.53589323, 2.60412104, 6.15349448, 1.96403106,
       2.19517056, 1.23346016, 5.3185532 , 5.38501   , 4.00282141,
       3.77793913, 2.51987386, 1.98087521, 2.95247283, 3.04782471,
       3.41367959, 3.07789941, 2.31556245, 2.85939093, 4.16083684,
       5.20250887, 3.48543535])

In [14]:
def print_weight(clf1, clf2=None):
    w = clf1.coef_.flatten()
    indices = np.argsort(w)[::-1]
    con = list(connotation.values())

    if clf2 is not None:
        w2 = clf2.coef_.flatten()
        for i in indices:
            print('%s \t %.3f \t %.3f \t %d' %(word_list[i], w[i], w2[i], con[i]))
    else:  
        for i in indices:
            print('%s \t %.3f \t %d' %(word_list[i], w[i],con[i]))

#### Train with predicted label

In [15]:
predicted_y = lr1.predict(X_train)
proba_y = lr1.predict_proba(X_train)[:,1]

In [16]:
lr2_train_bin = list()
for i,y in enumerate(predicted_y):
    if y == 1:
        lr2_train_bin.append(y_train_agreement_pos[i] * y)
    else:
        lr2_train_bin.append(y_train_agreement_neg[i] * y)
        
lr2_train_bin = np.asarray(lr2_train_bin)

lr2_train_proba = list()
for i,y in enumerate(proba_y):
    if y >= 0.5:
        lr2_train_proba.append(y_train_agreement_pos[i] * y)
    else:
        lr2_train_proba.append(y_train_agreement_neg[i] * y)
        
lr2_train_proba = np.asarray(lr2_train_proba)

In [17]:
# Train with {0,1} label
# Train with [0,1] proba

lr2_y_hat_bin = LogisticRegression(penalty='l1', random_state=42)
lr2_y_hat_proba = LogisticRegression(penalty='l1', random_state=42)

lr2_y_hat_bin.fit(lr2_train_bin, y_train)
lr2_y_hat_proba.fit(lr2_train_proba, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
y_test_pred_bin = lr2_y_hat_bin.predict(lr2_test_bin)
y_test_pred_proba = lr2_y_hat_proba.predict(lr2_test_proba)

In [19]:
print(lr2_y_hat_bin.score(lr2_test_bin,y_test))
print(lr2_y_hat_proba.score(lr2_test_proba,y_test))
print()
print(lr2_y_hat_bin.score(lr2_train_bin,y_train))
print(lr2_y_hat_proba.score(lr2_train_proba,y_train))

0.8821108334558283
0.8869616345729825

0.8837966985230234
0.8905299739357081


In [20]:
print(lr2_y_hat_bin.intercept_)
print(lr2_y_hat_proba.intercept_)

[0.57301472]
[0.5918641]


In [21]:
# print_weight(lr2_y_hat_bin, lr2_y_hat_proba)

In [22]:
# print_weight(lr2_y_hat_proba)

### What if... we put scale and ReLU manually

If we do not scale. Then, the negative weights would be closer to zero. The magnitude would be lesser.

In [23]:
predicted_y = lr1.predict(X_train)
proba_y = lr1.predict_proba(X_train)[:,1]

# Scale

predicted_y = predicted_y * 2 - 1
proba_y = proba_y * 2 - 1

In [24]:
lr2_train_bin = list()
for i,y in enumerate(predicted_y):
    if y == 1:
        lr2_train_bin.append(y_train_agreement_pos[i] * y)
    else:
        lr2_train_bin.append(y_train_agreement_neg[i] * y)
        
lr2_train_bin = np.asarray(lr2_train_bin)

lr2_train_proba = list()
for i,y in enumerate(proba_y):
    if y >= 0.5:
        lr2_train_proba.append(y_train_agreement_pos[i] * y)
    else:
        lr2_train_proba.append(y_train_agreement_neg[i] * y)
        
lr2_train_proba = np.asarray(lr2_train_proba)

In [25]:
# Train with {0,1} label
# Train with [0,1] proba

lr2_y_hat_bin = LogisticRegression(penalty='l1', random_state=42)
lr2_y_hat_proba = LogisticRegression(penalty='l1', random_state=42)

lr2_y_hat_bin.fit(lr2_train_bin, y_train)
lr2_y_hat_proba.fit(lr2_train_proba, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
# Test with predict {0,1}

predicted_y_test = lr1.predict(X_test)
predicted_y_test = predicted_y_test * 2 - 1

lr2_test_bin = list()
for i,y in enumerate(predicted_y_test):
    if y==1:
        lr2_test_bin.append(y_test_agreement_pos[i] * y)
    else:
        lr2_test_bin.append(y_test_agreement_neg[i] * y)
    
lr2_test_bin = np.asarray(lr2_test_bin)


# Test with predict [0,1]

predicted_y_test_proba = lr1.predict_proba(X_test)[:,1]
predicted_y_test_proba = predicted_y_test_proba * 2 - 1

lr2_test_proba = list()
for i,y in enumerate(predicted_y_test_proba):
    if y>=0:
        lr2_test_proba.append(y_test_agreement_pos[i] * y)
    else:
        lr2_test_proba.append(y_test_agreement_neg[i] * y)
    
lr2_test_proba = np.asarray(lr2_test_proba)

In [27]:
print(lr2_y_hat_bin.score(lr2_test_bin,y_test))
print(lr2_y_hat_proba.score(lr2_test_proba,y_test))
print()
print(lr2_y_hat_bin.score(lr2_train_bin,y_train))
print(lr2_y_hat_proba.score(lr2_train_proba,y_train))

0.9021020138174335
0.9015140379244451

0.9103677961193165
0.9112366058499856


In [28]:
accept_indices = np.where(np.sum(lr2_test_proba,axis=1)!=0)[0]

In [29]:
print(lr2_y_hat_bin.score(lr2_test_bin[accept_indices], y_test[accept_indices]))
print(lr2_y_hat_proba.score(lr2_test_proba[accept_indices], y_test[accept_indices]))

0.9530994364660971
0.9523722959461915


In [30]:
np.where(np.sum(lr2_test_bin,axis=1)==0)[0]

array([   3,   12,   24, ..., 6776, 6788, 6792], dtype=int64)

In [31]:
np.sum(np.sum(lr2_test_proba,axis=1)==0)/lr2_test_proba.shape[0]

0.19138615316772012

In [32]:
np.sum(np.sum(lr2_test_proba,axis=1)==0)

1302

In [33]:
lr2_test_proba[8]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.99999879, 0.        , 0.99999879, 0.        , 0.        ,
       0.        , 0.        , 0.99999879, 0.        , 0.99999879,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.99999879, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.99999879])

In [34]:
lr2_test_bin[8]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1])

In [35]:
predicted_y_test_proba[8]

0.999998786730796

In [36]:
predicted_y_test[8]

1

In [37]:
print_weight(lr2_y_hat_bin, lr2_y_hat_proba)

subtle 	 2.198 	 2.021 	 1
versatile 	 2.111 	 2.348 	 1
chic 	 2.081 	 1.798 	 1
glad 	 2.043 	 2.057 	 1
pleased 	 2.000 	 1.774 	 1
perfect 	 1.935 	 2.575 	 1
comfy 	 1.800 	 2.192 	 1
comfortable 	 1.668 	 2.245 	 1
complaint 	 1.615 	 2.065 	 1
stylish 	 1.574 	 1.515 	 1
elegant 	 1.440 	 1.426 	 1
great 	 1.418 	 2.018 	 1
stunning 	 1.406 	 2.290 	 1
happy 	 1.400 	 1.682 	 1
excellent 	 1.335 	 1.184 	 1
fits 	 1.328 	 1.681 	 1
amazing 	 1.308 	 1.524 	 1
fun 	 1.291 	 1.908 	 1
simple 	 1.260 	 1.666 	 1
roomy 	 1.249 	 2.365 	 1
feminine 	 1.246 	 1.431 	 1
hits 	 1.199 	 1.755 	 1
soft 	 1.164 	 1.799 	 1
loose 	 1.123 	 1.753 	 1
worried 	 1.088 	 1.417 	 1
cozy 	 1.072 	 1.311 	 1
fabulous 	 1.056 	 1.981 	 1
beautiful 	 1.030 	 1.794 	 1
nice 	 0.966 	 1.917 	 1
gorgeous 	 0.949 	 1.532 	 1
best 	 0.941 	 1.775 	 1
recommend 	 0.928 	 1.758 	 1
flattering 	 0.915 	 1.953 	 1
lovely 	 0.740 	 1.274 	 1
fantastic 	 0.573 	 1.430 	 1
bigger 	 0.214 	 0.792 	 1
oversized 	

In [38]:
# becasue LR sklearn use a regularization